In [2]:
import os
import numpy as np
import glob
from matplotlib import pyplot as plt

In [3]:
import rasterio
import earthpy.plot as ep
from patchify import patchify
import tifffile as tifffile
import random

In [4]:
os.getcwd()

'/data/private/Thesis/May'

**Validation**

In [5]:
## Read Sentinel-1 Polarimetric Covariance Matrix (C2): 3 components
with rasterio.open("Data/02_Tile/c2/c211.tif") as src:
    c11 = src.read(1)
    c12_r = src.read(2)
    c12_i = src.read(3)
    c22 = src.read(4)

# Convert c12 component to complex value
c12 = c12_r + 1j * c12_i
print("c12: " + str(c12.dtype))

c12: complex64


In [7]:
# Change order of the array, so the n channel is at axis -1
s1_img = np.transpose(s1_c, (1,2,0))
print(s1_img.shape)

(5903, 6055, 3)


In [6]:
# Append 0 as imaginary part of other components to make it complex-like
c11 = c11.astype(dtype=np.complex64)
c22 = c22.astype(dtype=np.complex64)

# Stack arrays
s1_c = np.stack((c11, c12, c22))
#s1_r = abs(s1_c)
#print(s1)
print(s1_c.shape)

(3, 5903, 6055)


In [8]:
# Define parameter
patch_size = 256
n_bands = s1_img.shape[-1]
print(n_bands)

3


In [ ]:
# Create patches for image

if not os.path.exists('val_c2'):
    os.makedirs('val_c2')

patches_img = patchify(s1_img, (patch_size, patch_size, n_bands), step=patch_size) #xdim, ydim, nband, step=xdim = no overlap
    
for i in range(patches_img.shape[0]):
    for j in range(patches_img.shape[1]):
        
        single_patch_img = patches_img[i,j,:,:]
        tifffile.imwrite('val_c2/' + 'patch_' + str(i) + '_' + str(j) + ".tif", single_patch_img) #str(img) if u have more than 1 source image to loop

In [11]:
image_patches = sorted(glob.glob('val_c2/*.tif'))
print(len(image_patches))

529


In [12]:
x = np.zeros((len(image_patches), patch_size, patch_size, n_bands), dtype='complex64') #NBANDS
for i, (img) in enumerate(zip(image_patches)):
    img = tifffile.imread(img)
    np_img = np.array(img)
    x[i] = np_img

In [13]:
np.save('Data/03_Patch/val/xc.npy', x)

**Training**

In [14]:
## Read Sentinel-1 Polarimetric Covariance Matrix (C2): 3 components
with rasterio.open("Data/02_Tile/c2/c200.tif") as src:
    c11 = src.read(1)
    c12_r = src.read(2)
    c12_i = src.read(3)
    c22 = src.read(4)

# Convert c12 component to complex value
c12 = c12_r + 1j * c12_i
print("c12: " + str(c12.dtype))

# Append 0 as imaginary part of other components to make it complex-like
c11 = c11.astype(dtype=np.complex64)
c22 = c22.astype(dtype=np.complex64)

# Stack arrays
s1_c_1 = np.stack((c11, c12, c22))
#s1_r = abs(s1_c)
#print(s1)
print(s1_c_1.shape)

c12: complex64
(3, 5903, 6055)


In [15]:
## Read Sentinel-1 Polarimetric Covariance Matrix (C2): 3 components
with rasterio.open("Data/02_Tile/c2/c201.tif") as src:
    c11 = src.read(1)
    c12_r = src.read(2)
    c12_i = src.read(3)
    c22 = src.read(4)

# Convert c12 component to complex value
c12 = c12_r + 1j * c12_i
print("c12: " + str(c12.dtype))

# Append 0 as imaginary part of other components to make it complex-like
c11 = c11.astype(dtype=np.complex64)
c22 = c22.astype(dtype=np.complex64)

# Stack arrays
s1_c_2 = np.stack((c11, c12, c22))
#s1_r = abs(s1_c)
#print(s1)
print(s1_c_2.shape)

c12: complex64
(3, 5903, 6055)


In [16]:
# Change order of the array, so the n channel is at axis -1
s1_img_1 = np.transpose(s1_c_1, (1,2,0))
s1_img_2 = np.transpose(s1_c_2, (1,2,0))
print(s1_img_1.shape)
print(s1_img_2.shape)

(5903, 6055, 3)
(5903, 6055, 3)


In [17]:
# Create patches for image

if not os.path.exists('Patch'):
    os.makedirs('Patch')

patches_img = patchify(s1_img_1, (patch_size, patch_size, n_bands), step=patch_size) #xdim, ydim, nband, step=xdim = no overlap
    
for i in range(patches_img.shape[0]):
    for j in range(patches_img.shape[1]):
        
        single_patch_img = patches_img[i,j,:,:]
        tifffile.imwrite('Patch/' + 'patchs00_' + str(i) + '_' + str(j) + ".tif", single_patch_img) #str(img) if u have more than 1 source image to loop

# Create patches for image

if not os.path.exists('Patch'):
    os.makedirs('Patch')

patches_img = patchify(s1_img_2, (patch_size, patch_size, n_bands), step=patch_size) #xdim, ydim, nband, step=xdim = no overlap
    
for i in range(patches_img.shape[0]):
    for j in range(patches_img.shape[1]):
        
        single_patch_img = patches_img[i,j,:,:]
        tifffile.imwrite('Patch/' + 'patchs01_' + str(i) + '_' + str(j) + ".tif", single_patch_img) #str(img) if u have more than 1 source image to loop

### Save image tiles to numpy format (useless as georeference loss)

In [18]:
image_patches = sorted(glob.glob('Patch/*.tif'))
print(len(image_patches))

1058


In [19]:
x = np.zeros((len(image_patches), patch_size, patch_size, n_bands), dtype='complex64') #NBANDS
for i, (img) in enumerate(zip(image_patches)):
    img = tifffile.imread(img)
    np_img = np.array(img)
    x[i] = np_img

In [20]:
#if not os.path.exists('Data/Patch_np'):
#    os.makedirs('Data/Patch_np')

#np.save('Data/02_Patches/Test/x1.npy', x)
#np.save('Data/02_Patches/Test/yb.npy', y)
#np.save('Data/03_Patch/train/xc_.npy', x)
np.save('Data/03_Patch/train/xc.npy', x)
#np.save('Data/y_compl.npy', y)